In [158]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [127]:
df = pd.read_csv('../BNB_nue data/nue_and_mu_unscaled.csv', index_col=0)
df_scaled = pd.read_csv('../BNB_nue data/nue_and_mu_scaled.csv', index_col=0)
df['envelope'] = df_scaled ['envelope']
df['eps']=df_scaled['eps']
df['RSS'] = df_scaled['RSS']
df = df.dropna()
df = df.loc[(df['completeness']>0.8) & (df['purity']>0.7) & (df['nHits']>4) ]
df.head()
pdg_dict={'p':2212,'mu-':13, 'gamma':22, 'pi0':11,'pi+':211, 'e':11 }

In [44]:
df_nue= pd.read_csv('../BNB_nue data/nue_uvw_scaled_properly.csv', index_col=0)
df_nue.head(2)

,isTrueTrack,pdg,completeness,purity,xu,zu,xv,zv,xw,zw,...,nHits,RSSu,RSSv,RSSw,envelopeu,envelopev,envelopew,lengthu,lengthv,lengthw
0,1,2212,1.000000,0.909091,"[119.256, 119.227, 118.956, 118.596, 118.286, ...","[452.06, 452.36, 452.66, 452.96, 453.26, 453.5...","[119.231, 119.332, 119.104, 118.985, 118.887, ...","[393.74, 394.04, 394.64, 394.94, 395.24, 395.5...","[119.384, 119.3, 119.172, 119.087, 119.007, 11...","[846.25, 846.55, 846.85, 847.15, 847.45, 847.7...",...,41,0.068989,0.019627,0.015505,8.780118,9.560003,9.579459,4.330168,4.704385,4.785806
1,0,11,0.892892,0.998911,"[164.312, 165.244, 164.521, 164.058, 163.652, ...","[264.26, 264.56, 264.86, 265.16, 265.46, 265.7...","[165.105, 165.067, 165.009, 164.947, 164.894, ...","[376.34, 376.64, 376.94, 377.24, 377.54, 377.8...","[165.12, 165.08, 165.017, 164.967, 164.921, 16...","[640.75, 641.05, 641.35, 641.65, 641.95, 642.2...",...,247,0.678124,0.208146,0.227368,15.270366,11.325856,15.046738,5.183616,5.209412,7.127999


In [45]:
variables = ['eps','RSS', 'envelope', 'sliding']
variablesu = [s + 'u' for s in variables]
variablesv = [s + 'v' for s in variables]
variablesw = [s + 'w' for s in variables]

In [46]:
df_nue = df_nue[['isTrueTrack', 'pdg', 'completeness', 'purity', 'nHits']+variablesw]
df_nue.columns=['isTrueTrack', 'pdg', 'completeness', 'purity', 'nHits']+variables
df_nue = df_nue.loc[(df_nue['completeness']>0.8) & (df_nue['purity']>0.7) & (df_nue['nHits']>4) ]
df_nue.head()

,isTrueTrack,pdg,completeness,purity,nHits,eps,RSS,envelope,sliding
0,1,2212,1.000000,0.909091,41,0.158,0.015505,9.579459,0.007796
1,0,11,0.892892,0.998911,247,2.351,0.227368,15.046738,0.112185
2,1,2212,0.989362,0.965398,136,0.531,0.051166,9.154247,0.004919
3,0,11,0.975904,0.852632,73,2.792,0.416976,18.236148,0.353822
4,1,2212,0.993789,0.977597,242,0.314,0.017907,10.112642,0.009616


In [199]:
event_df = pd.read_csv('../BNB_nue data/1004nuance_mu_p_gamma_gamma_vars.csv', index_col=0).tail(373*2)
event_df2 = pd.read_csv('../BNB_nue data/1003nuance_mu_p_pi+_vars.csv', index_col=0).tail(2000)
event_df.head(2)

,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event,x_scaled,z_scaled,eps,sliding,nHits,RSS,envelope
746,1,1004,2212,1.0,0.934783,"[151.193, 151.231, 151.251, 151.276, 151.296, ...","[548.05, 548.35, 548.65, 548.95, 549.25, 549.5...",23855,[-1.55080692 -1.53089346 -1.52041269 -1.507311...,[-1.72319115 -1.7054263 -1.68766144 -1.669896...,0.086,0.008536,195,0.042121,9.708579
747,1,1004,13,1.0,0.975845,"[151.16, 151.217, 151.303, 151.425, 151.571, 1...","[546.55, 546.85, 547.15, 547.45, 547.75, 548.0...",23855,[-1.41097782 -1.40075736 -1.38533701 -1.363461...,[-1.71247898 -1.67311165 -1.63374432 -1.594376...,0.117,0.017631,88,0.130699,9.758822


In [200]:
train_variables=variables+['nHits']
clf = GradientBoostingClassifier(max_depth=6, random_state=42)
train_df=pd.concat([df[df.isTrueTrack==0], df[df.isTrueTrack==1].head(1916)])
clf.fit(train_df[train_variables], train_df['isTrueTrack'])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)

In [201]:
y_pred = clf.predict(event_df[train_variables])
event_df['track_pred']=y_pred
event_df.head(2)

,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event,x_scaled,z_scaled,eps,sliding,nHits,RSS,envelope,track_pred
746,1,1004,2212,1.0,0.934783,"[151.193, 151.231, 151.251, 151.276, 151.296, ...","[548.05, 548.35, 548.65, 548.95, 549.25, 549.5...",23855,[-1.55080692 -1.53089346 -1.52041269 -1.507311...,[-1.72319115 -1.7054263 -1.68766144 -1.669896...,0.086,0.008536,195,0.042121,9.708579,1
747,1,1004,13,1.0,0.975845,"[151.16, 151.217, 151.303, 151.425, 151.571, 1...","[546.55, 546.85, 547.15, 547.45, 547.75, 548.0...",23855,[-1.41097782 -1.40075736 -1.38533701 -1.363461...,[-1.71247898 -1.67311165 -1.63374432 -1.594376...,0.117,0.017631,88,0.130699,9.758822,1


In [202]:
particles = ['mu-', 'p', 'gamma']
accuracy=[]
for pdg in particles:
    true = len(event_df[event_df.pdg==pdg_dict[pdg]])
    pred = len(event_df[(event_df.isTrueTrack==event_df.track_pred)][event_df.pdg==pdg_dict[pdg]])
    accuracy.append(pred/true)
print('CC_Nu_Mu nuance code:1004')
accuracy = [[i] for i in accuracy]
dictionary = dict(zip(particles, accuracy))
result = pd.DataFrame(data=dictionary)
result['prediction accuracy']= '%'
result =result.set_index('prediction accuracy')
result=result*100
result

CC_Nu_Mu nuance code:1004


,gamma,mu-,p
prediction accuracy,,,
%,84.408602,95.187166,96.256684


In [203]:
y_pred = clf.predict(event_df2[train_variables])
event_df2['track_pred']=y_pred
event_df2.head(2)

,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event,x_scaled,z_scaled,eps,sliding,nHits,RSS,envelope,track_pred
2875,1,1003,2212,0.981818,1.0,"[24.2038, 24.2778, 23.7412, 23.544, 23.3459, 2...","[19.45, 19.75, 20.05, 20.35, 20.65, 20.95, 21....",29916,[ 1.58011958 1.61917425 1.33597511 1.231899...,[-1.59879311 -1.48603188 -1.37327066 -1.260509...,0.318,0.013430,28,0.010981,9.247394,1
2876,1,1003,211,0.971831,1.0,"[25.2876, 25.8769, 27.3528, 26.4325, 27.5133, ...","[20.05, 20.35, 20.65, 20.65, 20.95, 20.95, 21....",29916,[-1.39218361 -1.23235074 -0.83204981 -1.081658...,[-1.48962359 -1.3331791 -1.17673462 -1.176734...,0.504,0.067318,23,0.133373,9.354366,1


In [204]:
particles = ['mu-', 'p', 'pi+']
accuracy=[]
for pdg in particles:
    true = len(event_df2[event_df2.pdg==pdg_dict[pdg]])
    pred = len(event_df2[(event_df2.isTrueTrack==event_df2.track_pred)][event_df2.pdg==pdg_dict[pdg]])
    accuracy.append(pred/true)
print('CC_Nu_Mu, nuance code:1003')
accuracy = [[i] for i in accuracy]
dictionary = dict(zip(particles, accuracy))
result = pd.DataFrame(data=dictionary)
result['prediction accuracy']= '%'
result =result.set_index('prediction accuracy')
result=result*100
result

CC_Nu_Mu, nuance code:1003


,mu-,p,pi+
prediction accuracy,,,
%,91.591592,95.202399,75.112444
